# Pulling BCN Open data from the API and processing

In [84]:
# This script retrieves the list of organizations from the Open Data API of Barcelona.
# It uses the requests library to make API calls and pandas for data manipulation.
import requests
import os
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

In [85]:
API_URL = "https://portaldades.ajuntament.barcelona.cat/services/backend/rest/statistic/export"

API_KEY = "e58ce2d31573401158da989c0906bb09"

headers = {'Authorization': API_KEY}

In [ ]:
import requests
import pandas as pd
from io import StringIO

# Constants
API_URL = "https://portaldades.ajuntament.barcelona.cat/services/backend/rest/statistic/export"
DATASET_ID = "?id=wc9hkmubl7&fileformat=CSV"
headers = {
    "X-IBM-Client-Id": "e58ce2d31573401158da989c0906bb09"
}

# Fetch CSV content
data_response = requests.get((API_URL + DATASET_ID), headers=headers)

# Load the CSV content into a pandas DataFrame
csv_file = StringIO(data_response.text)
df = pd.read_csv(csv_file, encoding='utf-8')

# Preview the data
print(df.head())


           Dim-00:TEMPS Dim-01:TERRITORI  Dim-01:TERRITORI (order)  \
0  2015-01-01T00:00:00Z        Barcelona                        -1   
1  2015-01-01T00:00:00Z        Barcelona                        -1   
2  2016-01-01T00:00:00Z        Barcelona                        -1   
3  2016-01-01T00:00:00Z        Barcelona                        -1   
4  2017-01-01T00:00:00Z        Barcelona                        -1   

  Dim-01:TERRITORI (type) Dim-02:TIPUS DE ETAPA      VALUE  
0                Municipi             ConnexiÃ³    80657.0  
1                Municipi               Interna  3280032.0  
2                Municipi             ConnexiÃ³    58655.0  
3                Municipi               Interna  2385291.0  
4                Municipi               Interna  2287812.0  


In [ ]:
import datetime
df[df.columns[0]] = pd.to_datetime(df['Dim-00:TEMPS'], utc=True)
df_grouped = df.groupby(df.columns[0]).sum()
df_grouped['VALUE'].plot(title='VALUE over Time')

Looks to be some sort of encoding error here but that's okay. We're just interested in the numbers for now.